In [1]:
!conda create --name botnet-detection python=3.10.9 -y
!conda activate botnet-detection

In [ ]:
!conda install -c conda-forge tensorflow=2.10.0 numpy=1.24.2 pandas=1.5.2 scikit-learn=1.1.2 sklearn-pandas=2.2.0 -y

In [91]:
import pandas as pd
import numpy as np
import tensorflow as tf

from pickle import load

tf.get_logger().setLevel('ERROR')

# Pandas-Jupyter options
pd.set_option("display.precision", 4)
pd.set_option("display.max_rows", 4)
pd.set_option("display.max_columns", 90)
pd.set_option("display.float_format", '{:,.2g}'.format)

# Disable SKLearn warnings
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

In [92]:
# Load scaler
scaler = load(open("scaler.pkl", 'rb'))

# Load model
model = tf.keras.models.load_model("dnn_2epochs.h5")
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16)                1168      
                                                                 
 dense_5 (Dense)             (None, 16)                272       
                                                                 
 dense_6 (Dense)             (None, 16)                272       
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,729
Trainable params: 1,729
Non-trainable params: 0
_________________________________________________________________


In [105]:
# Load flows
flows = pd.read_csv('flows/Selected_Flows.csv', index_col=0)
# flows = pd.read_csv('flows/examples/Virut/Virut_Flows.csv', index_col=0)
flows

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,FlowTerminationReason,FlowType,HostType,Predicted Probability,Predicted Type
1552,147.32.84.180-212.117.171.138-1445-65500-6,147.32.84.180,1445,212.117.171.138,65500,TCP,11/08/2011 11:42:06 AM,273832507,7,5,12,32,10,0,1.7,3.7,27,0,6.4,12,0.16,0.044,2.5e+07,7.5e+07,2.5e+08,1.1e+02,2.7e+08,4.6e+07,1e+08,2.5e+08,2.5e+02,2.5e+08,6.3e+07,1.2e+08,2.5e+08,2.2e+04,0,0,0,0,148,108,0.026,0.018,0,27,3.4,7.6,58,1,2,0,5,11,0,0,0,0,3.7,1.7,6.4,0,0,0,34,4,156,0,1,0,2,64240,5840,2,20,2.2e+05,2.7e+05,4.1e+05,2.2e+04,TCP_FIN,Neris,INFECTED,0.99,MALICIOUS
1989,147.32.84.180-212.117.171.138-3866-65500-6,147.32.84.180,3866,212.117.171.138,65500,TCP,11/08/2011 11:48:45 AM,231726551,8,5,12,32,10,0,1.5,3.5,27,0,6.4,12,0.19,0.056,1.9e+07,5.7e+07,2e+08,2.3e+02,2.3e+08,3.3e+07,7.4e+07,2e+08,1.2e+04,2.1e+08,5.2e+07,9.9e+07,2e+08,2.1e+04,0,0,0,0,168,108,0.035,0.022,0,27,3.1,7.4,55,1,2,0,5,12,0,0,0,0,3.4,1.5,6.4,0,0,0,0,0,0,0,0,0,2,64240,5840,2,20,2e+05,1.7e+05,3.5e+05,2.3e+04,TCP_FIN,Neris,INFECTED,0.99,MALICIOUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314380,147.32.84.180-46.4.36.120-3397-443-6,147.32.84.180,3397,46.4.36.120,443,TCP,11/08/2011 01:45:03 PM,34247937,12,12,0,0,0,0,0,0,0,0,0,0,0,0.7,1.5e+06,3.4e+06,1.1e+07,2.7e+04,3.4e+07,3.1e+06,4.5e+06,1.1e+07,5e+05,3.4e+07,3.1e+06,4.5e+06,1.1e+07,5e+05,0,0,0,0,336,240,0.35,0.35,0,0,0,0,0,0,12,12,0,12,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,64240,0,0,28,1e+06,1.8e+03,1e+06,1e+06,UNKNOWN,Neris,INFECTED,1,MALICIOUS
314776,147.32.84.180-46.4.36.120-4502-443-6,147.32.84.180,4502,46.4.36.120,443,TCP,11/08/2011 02:44:47 PM,15878032,9,9,0,0,0,0,0,0,0,0,0,0,0,1.1,9.3e+05,2.1e+06,6.9e+06,2.7e+04,1.6e+07,2e+06,2.8e+06,6.9e+06,5e+05,1.6e+07,2e+06,2.8e+06,6.9e+06,5e+05,0,0,0,0,252,180,0.57,0.57,0,0,0,0,0,0,9,9,0,9,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,64240,0,0,28,1e+06,4.2e+02,1e+06,1e+06,UNKNOWN,Neris,INFECTED,1,MALICIOUS


In [106]:
# Column names which are not used as features
non_features = [
    "Flow ID", "Src IP", "Src Port", "Dst IP", "Dst Port", 
    "Timestamp", "HostType", "FlowType", "Protocol", "FlowTerminationReason",
    "Predicted Probability", "Predicted Type",
    "Idle Mean", "Idle Std", "Idle Max", "Idle Min"]
feature_filter = ~flows.columns.isin(non_features)
# Use df.loc[:,feature_filter] to get the features only

In [107]:
def loss_gradient(input, ground_truth):
    loss_object = tf.keras.losses.BinaryCrossentropy()

    with tf.GradientTape() as tape:
      tape.watch(input)
      prediction = model(input) # "model" must be initialized earlier
      loss = loss_object(ground_truth, prediction)

    # Get the gradients of the loss w.r.t to the input
    gradient = tape.gradient(loss, input)
    gradient_sign = tf.sign(gradient)
    return gradient.numpy()[0], gradient_sign.numpy()[0]

In [108]:
def flow_gradients(flows):
    df_grad, df_grad_signs = flows.copy(), flows.copy()     # Leave original frame intact
    features_df = df_grad.loc[:, feature_filter]            # Leave features only
    features = scaler.transform(features_df.to_numpy())     # Scaled features

    # Calculate loss function gradients for each sample
    for i in range(len(df_grad)):
        gradient, gradient_sign = loss_gradient(tf.cast([features[i]], tf.float32), [[1]]) # Label 1 imlies flows are malicious
        df_grad.iloc[i, feature_filter] = gradient
        df_grad_signs.iloc[i, feature_filter] = gradient_sign
    return df_grad, df_grad_signs

flows_grad, flows_grad_signs = flow_gradients(flows)
flows_grad_signs

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,FlowTerminationReason,FlowType,HostType,Predicted Probability,Predicted Type
1552,147.32.84.180-212.117.171.138-1445-65500-6,147.32.84.180,1445,212.117.171.138,65500,TCP,11/08/2011 11:42:06 AM,1,1,1,1,1,-1,-1,1,1,-1,-1,1,1,1,-1,1,1,1,1,-1,1,1,-1,1,1,-1,-1,1,-1,1,1,-1,-1,1,1,-1,1,-1,1,-1,1,1,1,-1,-1,-1,1,-1,-1,-1,1,1,1,1,1,-1,-1,1,1,-1,-1,-1,-1,1,-1,-1,1,1,1,1,-1,1,TCP_FIN,Neris,INFECTED,0.99,MALICIOUS
1989,147.32.84.180-212.117.171.138-3866-65500-6,147.32.84.180,3866,212.117.171.138,65500,TCP,11/08/2011 11:48:45 AM,1,1,1,1,1,-1,-1,1,1,-1,-1,1,1,1,-1,1,1,1,1,-1,1,1,-1,1,1,-1,-1,1,-1,1,1,-1,-1,1,1,-1,1,-1,1,-1,1,1,1,-1,-1,-1,1,-1,-1,-1,1,1,1,1,1,-1,-1,1,1,-1,-1,-1,-1,1,-1,-1,1,1,1,1,1,1,TCP_FIN,Neris,INFECTED,0.99,MALICIOUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314380,147.32.84.180-46.4.36.120-3397-443-6,147.32.84.180,3397,46.4.36.120,443,TCP,11/08/2011 01:45:03 PM,1,1,1,1,1,-1,-1,-1,-1,-1,1,1,-1,1,-1,1,1,1,-1,-1,1,1,-1,1,1,-1,-1,1,-1,1,1,-1,-1,1,1,-1,1,-1,-1,-1,1,-1,1,-1,-1,-1,1,1,-1,-1,1,1,-1,1,1,1,1,-1,-1,-1,-1,-1,-1,1,-1,1,1,1,1,1,1,1,UNKNOWN,Neris,INFECTED,1,MALICIOUS
314776,147.32.84.180-46.4.36.120-4502-443-6,147.32.84.180,4502,46.4.36.120,443,TCP,11/08/2011 02:44:47 PM,1,1,1,1,1,-1,-1,-1,-1,-1,1,1,1,1,-1,1,1,1,1,1,1,1,-1,1,1,-1,-1,1,-1,1,1,-1,-1,1,1,-1,1,-1,1,-1,1,-1,1,-1,-1,-1,1,1,-1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,1,1,1,1,1,1,UNKNOWN,Neris,INFECTED,1,MALICIOUS


In [109]:
# make flows adversarial (or at least try)
def make_adversarial(flows, flows_grad_signs):
    features_to_change = [
        "Flow Duration", 
        "Flow IAT Mean", "Flow IAT Std", "Flow IAT Max", "Flow IAT Min", 
        "Fwd IAT Total", "Fwd IAT Mean", "Fwd IAT Std", "Fwd IAT Max", "Fwd IAT Min", 
        "Bwd IAT Total", "Bwd IAT Mean", "Bwd IAT Std", "Bwd IAT Max", "Bwd IAT Min",
    ]
    # Get only the features we want to change
    feautre_grad_signs = flows_grad_signs.loc[:, feature_filter]
    # Set all gradients to zero except the ones we want to change
    feautre_grad_signs.loc[:, ~feautre_grad_signs.columns.isin(features_to_change)] = 0
    
    # Note: Epsilon-scaling is applied BEFORE MinMax-scaling
    eps = [ 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7 ]
    result = {}
    print("Misclassification rate:")
    for eps in eps:
        df = flows.copy()
        f = df.loc[:, feature_filter]
        # Scale feature in the direction of the gradient
        f = f + eps * f * feautre_grad_signs
        
        # Recalculate dependant features
        # Note: Flow Duration is in microseconds
        f["Fwd Packets/s"] = f["Total Fwd Packet"] / f["Flow Duration"] * 10**6
        f["Bwd Packets/s"] = f["Total Bwd packets"] / f["Flow Duration"] * 10**6
        f["Flow Packets/s"] = f["Fwd Packets/s"] + f["Bwd Packets/s"]
        f["Flow Bytes/s"] = (f["Total Length of Fwd Packet"] + f["Total Length of Bwd Packet"] ) / f["Flow Duration"] * 10**6

        new_predictions = model.predict(scaler.transform(f), verbose=0)
        df.rename(columns={"Predicted Probability": "Predicted Probability (original)"}, inplace=True)
        df.rename(columns={"Predicted Type": "Predicted Type (original)"}, inplace=True)

        df["Predicted Probability (epsilon = " + str(eps) + ")"] = new_predictions
        print(f'eps = {str(eps)}: {np.sum(new_predictions < 0.5) / len(new_predictions) * 100:.2f}%')
        result[eps] = df

    return result

result = make_adversarial(flows, flows_grad_signs)

Misclassification rate:
eps = 0.1: 0.00%
eps = 0.2: 1.25%
eps = 0.3: 7.48%
eps = 0.4: 10.97%
eps = 0.5: 14.46%
eps = 0.6: 20.20%
eps = 0.7: 24.94%


In [110]:
result[0.5]

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,FlowTerminationReason,FlowType,HostType,Predicted Probability (original),Predicted Type (original),Predicted Probability (epsilon = 0.5)
1552,147.32.84.180-212.117.171.138-1445-65500-6,147.32.84.180,1445,212.117.171.138,65500,TCP,11/08/2011 11:42:06 AM,273832507,7,5,12,32,10,0,1.7,3.7,27,0,6.4,12,0.16,0.044,2.5e+07,7.5e+07,2.5e+08,1.1e+02,2.7e+08,4.6e+07,1e+08,2.5e+08,2.5e+02,2.5e+08,6.3e+07,1.2e+08,2.5e+08,2.2e+04,0,0,0,0,148,108,0.026,0.018,0,27,3.4,7.6,58,1,2,0,5,11,0,0,0,0,3.7,1.7,6.4,0,0,0,34,4,156,0,1,0,2,64240,5840,2,20,2.2e+05,2.7e+05,4.1e+05,2.2e+04,TCP_FIN,Neris,INFECTED,0.99,MALICIOUS,0.29
1989,147.32.84.180-212.117.171.138-3866-65500-6,147.32.84.180,3866,212.117.171.138,65500,TCP,11/08/2011 11:48:45 AM,231726551,8,5,12,32,10,0,1.5,3.5,27,0,6.4,12,0.19,0.056,1.9e+07,5.7e+07,2e+08,2.3e+02,2.3e+08,3.3e+07,7.4e+07,2e+08,1.2e+04,2.1e+08,5.2e+07,9.9e+07,2e+08,2.1e+04,0,0,0,0,168,108,0.035,0.022,0,27,3.1,7.4,55,1,2,0,5,12,0,0,0,0,3.4,1.5,6.4,0,0,0,0,0,0,0,0,0,2,64240,5840,2,20,2e+05,1.7e+05,3.5e+05,2.3e+04,TCP_FIN,Neris,INFECTED,0.99,MALICIOUS,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314380,147.32.84.180-46.4.36.120-3397-443-6,147.32.84.180,3397,46.4.36.120,443,TCP,11/08/2011 01:45:03 PM,34247937,12,12,0,0,0,0,0,0,0,0,0,0,0,0.7,1.5e+06,3.4e+06,1.1e+07,2.7e+04,3.4e+07,3.1e+06,4.5e+06,1.1e+07,5e+05,3.4e+07,3.1e+06,4.5e+06,1.1e+07,5e+05,0,0,0,0,336,240,0.35,0.35,0,0,0,0,0,0,12,12,0,12,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,64240,0,0,28,1e+06,1.8e+03,1e+06,1e+06,UNKNOWN,Neris,INFECTED,1,MALICIOUS,1
314776,147.32.84.180-46.4.36.120-4502-443-6,147.32.84.180,4502,46.4.36.120,443,TCP,11/08/2011 02:44:47 PM,15878032,9,9,0,0,0,0,0,0,0,0,0,0,0,1.1,9.3e+05,2.1e+06,6.9e+06,2.7e+04,1.6e+07,2e+06,2.8e+06,6.9e+06,5e+05,1.6e+07,2e+06,2.8e+06,6.9e+06,5e+05,0,0,0,0,252,180,0.57,0.57,0,0,0,0,0,0,9,9,0,9,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,64240,0,0,28,1e+06,4.2e+02,1e+06,1e+06,UNKNOWN,Neris,INFECTED,1,MALICIOUS,1
